In [1]:
import os
import glob
import numpy as np
from dotenv import load_dotenv
from pymilvus import MilvusClient, DataType
from oci.auth.signers import InstancePrincipalsSecurityTokenSigner
from oci.generative_ai_inference.generative_ai_inference_client import GenerativeAiInferenceClient
from oci.generative_ai_inference.models import EmbedTextDetails, OnDemandServingMode
from langchain_text_splitters import CharacterTextSplitter

In [2]:
load_dotenv()
uri = os.getenv("MILVUS_URI")
compartment_id = os.getenv("COMPARTMENT_ID")
endpoint = os.getenv("GENAI_ENDPOINT")
embedding_batch_size=96
collection_name="ochacafe_collection"
client = MilvusClient(uri=uri)
generative_ai_inference_client = GenerativeAiInferenceClient(config={}, signer=InstancePrincipalsSecurityTokenSigner(), service_endpoint=endpoint)

In [ ]:
client.describe_collection(collection_name=collection_name)
client.drop_collection(collection_name=collection_name)

In [4]:
schema = MilvusClient.create_schema(
    auto_id=True,
    enable_dynamic_field=True,
)

schema.add_field(field_name="id", datatype=DataType.INT64, is_primary=True)
schema.add_field(field_name="vector", datatype=DataType.FLOAT_VECTOR, dim=1024)

client.create_collection(
    collection_name=collection_name,
    dimension=1024,
    schema=schema
)

In [5]:
def embedding(inputs: list[str]) -> list[float]:
    res = generative_ai_inference_client.embed_text(
        embed_text_details=EmbedTextDetails(
            compartment_id=compartment_id,
            inputs=inputs,
            input_type="SEARCH_DOCUMENT",
            serving_mode=OnDemandServingMode(
                model_id="cohere.embed-multilingual-v3.0"
            )
        )
    )
    if res.status != 200:
        print("Something went wrong.")
        exit()
    embeddings = res.data.embeddings
    return embeddings

In [6]:
def create_data_by_character_text_splitter(content: str) -> list[dict]:
    text_splitter = CharacterTextSplitter(
        separator="\n",
        chunk_size=500,
        chunk_overlap=200,
        length_function=len,
        is_separator_regex=False,
    )
    texts = text_splitter.create_documents([content])
    inputs = [text.page_content for text in texts]
    if len(inputs) > embedding_batch_size:
        vectors = []
        inputs_ary = np.array_split(inputs, len(inputs)//embedding_batch_size + 1)
        for input in inputs_ary:
            vectors.extend(embedding(inputs=input.tolist()))
    else:
        vectors = embedding(inputs=inputs)
    data = [
        {"vector": vectors[i], "text": inputs[i], "subject": "technology"} # enabled auto-id
        for i in range(len(vectors))
    ]
    return data

In [7]:
def insert_data(data: dict | list[dict]) -> dict:
    res = client.insert(
        collection_name=collection_name,
        data=data
    )
    return res

In [8]:
files = glob.glob("../data/text/*.txt")
for file in files:
    with open(file) as f:
        content = f.read()
        data = create_data_by_character_text_splitter(content)
        _ = insert_data(data)

Created a chunk of size 673, which is longer than the specified 500
Created a chunk of size 673, which is longer than the specified 500
Created a chunk of size 673, which is longer than the specified 500
Created a chunk of size 673, which is longer than the specified 500


In [9]:
index_params = MilvusClient.prepare_index_params()

index_params.add_index(
    field_name="vector",
    metric_type="COSINE",
    index_type="IVF_FLAT",
    index_name="vector_index",
    params={ "nlist": 128 }
)

client.create_index(
    collection_name=collection_name,
    index_params=index_params
)

In [11]:
client.load_collection(collection_name=collection_name)
res = client.get_load_state(collection_name=collection_name)
print("load state: ", res)

load state:  {'state': <LoadState: Loaded>}


In [12]:
query_vectors = embedding(["OCHaCafeとはなんですか？"])

res = client.search(
    collection_name="ochacafe_collection",
    data=query_vectors,
    limit=3,
    output_fields=["text", "subject"]
)

print(res)

data: ["[{'id': 450394403568093342, 'distance': 0.615086555480957, 'entity': {'text': '「Oracle Cloud Hangout Cafe」コミュニティとは\\n「Oracle Cloud Hangout Cafe」(通称「おちゃかふぇ」/以降、OCHaCafe)は、日本オラクルが主催するコミュニティの1つです。定期的に、開発者・エンジニアに向けたクラウドネイティブな時代に身につけておくべきテクノロジーを深堀する勉強会を開催しています。\\nOCHaCafeのテーマは、Oracleの製品や技術に特化せず、オープン/デファクト・スタンダードを基準に選定して、参加される方々がいかなるプラットフォームを利用している場合でも、必ずスキルアップに役立てられる内容を趣旨としています。\\n本連載について\\nOCHaCafeは、2018年12月から開始して、この4年間、多くの方にご参加いただきました。そして、この4年の間にたくさんの有益なコンテンツが蓄積されました。本連載では、その中からご好評いただいたテーマを選択して、ダイジェスト(全6回)としてお送りします。', 'subject': 'technology'}}, {'id': 450394403568093324, 'distance': 0.615086555480957, 'entity': {'text': '「Oracle Cloud Hangout Cafe」コミュニティとは\\n「Oracle Cloud Hangout Cafe」(通称「おちゃかふぇ」/以降、OCHaCafe)は、日本オラクルが主催するコミュニティの1つです。定期的に、開発者・エンジニアに向けたクラウドネイティブな時代に身につけておくべきテクノロジーを深堀する勉強会を開催しています。\\nOCHaCafeのテーマは、Oracleの製品や技術に特化せず、オープン/デファクト・スタンダードを基準に選定して、参加される方々がいかなるプラットフォームを利用している場合でも、必ずスキルアップに役立てられる内容を趣旨としています。\\n本連載について\\nOCHaCafeは、2018年12月から開始して、この4年間、多くの方にご参加いただきました。そして、この4年の